In [ ]:
import torch
#assert '.'.join(torch.__version__.split('.')[:2]) == '1.4'
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

import torch.nn.functional as F  # useful stateless functions

import numpy as np

Code snippet for you to start with; modified from CS231 and the Pytorch Tutorial

In [ ]:
NUM_TRAIN = 49000

# The torchvision.transforms package provides tools for preprocessing data
# and for performing data augmentation; here we set up a transform to
# preprocess the data by subtracting the mean RGB value and dividing by the
# standard deviation of each RGB value; we've hardcoded the mean and std.
transform = T.Compose([
                T.ToTensor(),
                T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
            ])

cifar10_train = dset.CIFAR10('./datasets', train=True, download=True,
                             transform=transform)
loader_train = DataLoader(cifar10_train, batch_size=64, 
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar10_val = dset.CIFAR10('./datasets', train=True, download=True,
                           transform=transform)
loader_val = DataLoader(cifar10_val, batch_size=64, 
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

cifar10_test = dset.CIFAR10('./datasets', train=False, download=True, 
                            transform=transform)
loader_test = DataLoader(cifar10_test, batch_size=64)


In [ ]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

Here is code for a two-layer network; modify it in a new cell to create your own three and four layer network (no homework)

In [ ]:
def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)


In [ ]:
class TwoLayerFC(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        # http://pytorch.org/docs/master/nn.html#torch-nn-init 
        nn.init.kaiming_normal_(self.fc1.weight)
        self.fc2 = nn.Linear(hidden_size, num_classes)
        nn.init.kaiming_normal_(self.fc2.weight)
    
    def forward(self, x):
        x = flatten(x)
        scores = self.fc2(F.relu(self.fc1(x)))
        return scores


In [ ]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [ ]:
def train(model, optimizer, epochs=5):
    """
    Returns nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update!!!!!!!!
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % 200 == 0:
                print('Epoch %d, Iteration %d, loss = %.4f' % (e,t, loss.item()))
                check_accuracy(loader_val, model)
                print()
        check_accuracy(loader_test,model)
        print()

In [ ]:
hidden_layer_size = 4000
learning_rate = 1e-2
model = TwoLayerFC(3 * 32 * 32, hidden_layer_size, 10)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

train(model, optimizer, epochs = 20)

Here is code for a small conv net; modify it to get higher accuracy (Problem 2.) Try padding etc.

In [ ]:
class SmallConv(nn.Module):

    def __init__(self,input_size, num_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 3)
        self.conv1b = nn.Conv2d(6, 10, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(10, 13, 3)
        self.conv2b = nn.Conv2d(13, 16, 3)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 80)
        self.fc3 = nn.Linear(80, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv1b(x)))
        x = F.relu(self.conv2(x))
        x = self.pool(F.relu(self.conv2b(x)))
        x = flatten(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [ ]:
learning_rate = 1e-3
model = SmallConv(3 * 32 * 32, 10)
optimizer = optim.SGD(model.parameters(), lr=learning_rate,momentum=0.9)

train(model, optimizer,epochs=40)

Code to start with for transfer learning

In [ ]:
from torchvision import models

#Use pretrained AlexNet
alexnet = models.alexnet(pretrained=True)
print(alexnet)

#modify sizes of fully connected
alexnet.classifier[4] = nn.Linear(4096,1024)
alexnet.classifier[6] = nn.Linear(1024,10)
print(alexnet)

In [ ]:

transform_alex = T.Compose([
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

cifar10_train_alex = dset.CIFAR10('./datasets', train=True, download=True,
                             transform=transform_alex)
loader_train = DataLoader(cifar10_train_alex, batch_size=64, 
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar10_val_alex = dset.CIFAR10('./datasets', train=True, download=True,
                           transform=transform_alex)
loader_val = DataLoader(cifar10_val_alex, batch_size=64, 
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

cifar10_test_alex = dset.CIFAR10('./datasets', train=False, download=True, 
                            transform=transform_alex)
loader_test = DataLoader(cifar10_test_alex, batch_size=64)



In [ ]:
learning_rate = 1e-3
optimizer = optim.SGD(alexnet.parameters(), lr=learning_rate,momentum=0.9)
train(alexnet, optimizer,epochs=15)

In [ ]:
mobilenet = models.mobilenet_v2(pretrained=True)
print(mobilenet)



mobilenet.classifier[1] = nn.Sequential(
    nn.Linear(1280, 128),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(128, 10), 
)

print(mobilenet)

In [ ]:
learning_rate = 1e-3
optimizer = optim.SGD(resnet18.parameters(), lr=learning_rate,momentum=0.9)
train(resnet18, optimizer,epochs=15)

In [ ]:
resnet18 = models.resnet18(pretrained=True)
print(resnet18)

fc_inputs = resnet18.fc.in_features

resnet18.fc = nn.Sequential(
    nn.Linear(fc_inputs, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, 10), 
    nn.LogSoftmax(dim=1) # For using NLLLoss()
)
print(resnet18)


In [ ]:
learning_rate = 1e-3
optimizer = optim.SGD(mobilenet.parameters(), lr=learning_rate,momentum=0.9)
train(mobilenet, optimizer,epochs=15)